In [19]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import os
from urllib.request import urlopen,urlretrieve
from PIL import Image
from tqdm import tqdm_notebook
%matplotlib inline
from sklearn.utils import shuffle
import cv2
from resnets_utils import *

from tensorflow.keras.models import load_model
from sklearn.datasets import load_files   
from keras.utils import np_utils
from glob import glob
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential,Model,load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
from tensorflow.keras.callbacks import TensorBoard,ReduceLROnPlateau,ModelCheckpoint


### Load the data set

In [20]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

# Normalize image vectors
X_train = X_train_orig/255.
X_test = X_test_orig/255.

# Convert training and test labels to one hot matrices
Y_train = convert_to_one_hot(Y_train_orig, 6).T
Y_test = convert_to_one_hot(Y_test_orig, 6).T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 1080
number of test examples = 120
X_train shape: (1080, 64, 64, 3)
Y_train shape: (1080, 6)
X_test shape: (120, 64, 64, 3)
Y_test shape: (120, 6)


In [21]:
img_height,img_width = 64,64 
num_classes = 6
#If imagenet weights are being loaded, 
#input must have a static square shape (one of (128, 128), (160, 160), (192, 192), or (224, 224))
base_model = applications.resnet50.ResNet50(weights= None, include_top=False, input_shape= (img_height,img_width,3))

In [22]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.7)(x)
predictions = Dense(num_classes, activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)

In [24]:
from keras.optimizers import SGD, Adam
# sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
adam = Adam(lr=0.0001)
model.compile(optimizer= 'adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [25]:
model.fit(X_train, Y_train, epochs = 100, batch_size = 64)

Epoch 1/100
1080/1080 [==============================] - 73s 68ms/sample - loss: 2.9297 - accuracy: 0.2583
Epoch 2/100
1080/1080 [==============================] - 72s 66ms/sample - loss: 1.3616 - accuracy: 0.4991
Epoch 3/100
1080/1080 [==============================] - 71s 65ms/sample - loss: 0.5822 - accuracy: 0.7657
Epoch 4/100
1080/1080 [==============================] - 70s 65ms/sample - loss: 0.2095 - accuracy: 0.9176
Epoch 5/100
1080/1080 [==============================] - 68s 63ms/sample - loss: 0.2020 - accuracy: 0.9389
Epoch 6/100
1080/1080 [==============================] - 70s 65ms/sample - loss: 0.1479 - accuracy: 0.9435
Epoch 7/100
1080/1080 [==============================] - 66s 61ms/sample - loss: 0.1541 - accuracy: 0.9472
Epoch 8/100
 768/1080 [====================>.........] - ETA: 19s - loss: 0.1354 - accuracy: 0.9622

KeyboardInterrupt: 

In [ ]:
preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

In [ ]:
model.summary()